This notebook shows how crossreferences can be explored in python.

As a preparatory step you need to install "crossrefapi":

> pip install crossrefapi

A good introduction demo is https://pypi.org/project/crossrefapi/1.0.3/, here we focus at the main funcionality.


To start with we set up the environment.

In [36]:
from crossref.restful import Works
works = Works()

Choose a DOI and retrive the article information.

Here you might change to an "own" DOI





In [50]:


DOI = '10.1017/jfm.2017.541'
DOI = '10.1007/978-94-007-0910-2_18' # Stillman; book !
DOI = '10.1007/s10463-019-00720-8'
DOI = '10.1016/j.chb.2019.01.037' # Natalia
# DOI = '10.22937/ijcsns.2022.22.4.37' # Felipe
doi = DOI
dct = works.doi(DOI)



An readable output format is as a json object.

In [51]:
import json
print(json.dumps(dct, sort_keys=True, indent=4))

{
    "DOI": "10.1016/j.chb.2019.01.037",
    "ISSN": [
        "0747-5632"
    ],
    "URL": "http://dx.doi.org/10.1016/j.chb.2019.01.037",
    "alternative-id": [
        "S0747563219300469"
    ],
    "assertion": [
        {
            "label": "This article is maintained by",
            "name": "publisher",
            "value": "Elsevier"
        },
        {
            "label": "Article Title",
            "name": "articletitle",
            "value": "Digital hoarding behaviours: Measurement and evaluation"
        },
        {
            "label": "Journal Title",
            "name": "journaltitle",
            "value": "Computers in Human Behavior"
        },
        {
            "label": "CrossRef DOI link to publisher maintained version",
            "name": "articlelink",
            "value": "https://doi.org/10.1016/j.chb.2019.01.037"
        },
        {
            "label": "Content Type",
            "name": "content_type",
            "value": "article"
        },
 

E.g. we want to look up some author information:

In [52]:
authors = dct.get('author')
print(len(authors))
authors[0]



4


{'given': 'Nick', 'family': 'Neave', 'sequence': 'first', 'affiliation': []}

Get the abstract.

In [53]:
abstract = dct.get('abstract')
print(abstract)

None


In [54]:
def get_data(doi):
    dct = works.doi(doi)
    url = dct.get('URL')
    cited_count = dct.get('is-referenced-by-count')
    ref_count = dct.get('reference-count')
    title = dct.get('title')
    abstract = dct.get('abstract')
    print('title:', title, '\n ref_cout:', ref_count, '; cited_count:', cited_count, ';\n abstract:', abstract, '\n\n')
    
    
# doi = '10.1007/s10463-019-00720-8'
get_data(doi) 

title: ['Digital hoarding behaviours: Measurement and evaluation'] 
 ref_cout: 24 ; cited_count: 11 ;
 abstract: None 




Get the references

In [55]:
references = dct.get('reference')
len(references)

24

In [56]:
references[4]

{'key': '10.1016/j.chb.2019.01.037_bib5',
 'doi-asserted-by': 'crossref',
 'first-page': '179',
 'DOI': '10.1016/S0005-7967(01)00136-X',
 'article-title': 'Hoarding behaviours in a large college sample',
 'volume': '41',
 'author': 'Coles',
 'year': '2003',
 'journal-title': 'Behaviour Research and Therapy'}

In [57]:
references[4].get('DOI')

'10.1016/S0005-7967(01)00136-X'

Looping over the references we can display the DOI of each reference.

In [58]:
for k in range(0, len(references)):
    ref_doi = references[k].get('DOI')
    print(k, ': ', ref_doi)

0 :  None
1 :  10.1002/asi.10283
2 :  None
3 :  None
4 :  10.1016/S0005-7967(01)00136-X
5 :  10.1002/meet.2011.14504801304
6 :  10.1002/asi.22864
7 :  None
8 :  None
9 :  10.1287/orsc.2014.0945
10 :  10.1016/0005-7967(93)90094-B
11 :  None
12 :  10.1007/s10862-005-3265-z
13 :  None
14 :  10.1016/0003-6870(88)90199-8
15 :  10.1192/bjp.bp.113.130195
16 :  10.21153/ps2015vol1no2art472
17 :  None
18 :  10.1016/j.cpr.2003.08.002
19 :  10.1023/A:1025428631552
20 :  10.1016/j.chb.2018.03.031
21 :  10.1002/jclp.20795
22 :  10.1016/j.psychres.2009.05.001
23 :  None


Recall the possibility to download a "pdf" file by sci-hub, for a known DOI:

In [11]:
from scidownl import scihub_download

# paper = "https://doi.org/10.1017/jfm.2017.541"
# doi = "10.1017/jfm.2017.541"
out = "./paper/"
# scihub_download(doi, paper_type="doi")
scihub_download(doi, out=out)

[INFO] | 2022/09/21 15:43:26 | Choose scihub url [0]: http://sci-hub.se
[INFO] | 2022/09/21 15:43:26 | <- Request: scihub_url=http://sci-hub.se, source=DoiSource[type=doi, id=10.1017/jfm.2017.541]
[INFO] | 2022/09/21 15:43:26 | -> Response: status_code=200, content_length=7091
[INFO] | 2022/09/21 15:43:26 | * Extracted information: {'url': 'https://zero.sci-hub.se/6541/767bbda4d809eb8d01cbceed9673512f/grassia2017.pdf', 'title': 'Foam front advance during improved oil recovery  similarity solutions at early times near the top of the front. Journal of Fluid Mechanics, 828, 527–572'}
[INFO] | 2022/09/21 15:43:26 | ↓ Successfully download the url to: ./paper/Foam front advance during improved oil recovery  similarity solutions at early times near the top of the front. Journal of Fluid Mechanics, 828, 527–572.pdf


100% [........................................................] 664669 / 664669


Now we can download all pdf files where the reference contains a DOI.
We have to weed out the cases where a reference does not have a DOI.

In [60]:
for k in range(0, len(references)):	
    ref_doi=references[k].get('DOI')
    if ref_doi != "None":
        if(ref_doi):
            print(k, ref_doi)
            scihub_download(ref_doi, out=out)
        else:
            print("---------------------------")

[INFO] | 2022/09/21 16:44:09 | Choose scihub url [0]: http://sci-hub.se


---------------------------
1 10.1002/asi.10283


[INFO] | 2022/09/21 16:44:09 | <- Request: scihub_url=http://sci-hub.se, source=DoiSource[type=doi, id=10.1002/asi.10283]
[INFO] | 2022/09/21 16:44:09 | -> Response: status_code=200, content_length=7088
[INFO] | 2022/09/21 16:44:09 | * Extracted information: {'url': 'https://moscow.sci-hub.se/62/1ca0dc0860f5158a5cf682feba6f3800/bergman2003.pdf', 'title': 'The user-subjective approach to personal information management systems. Journal of the American Society for Information Science and Technology, 54(9), 872–878'}
[INFO] | 2022/09/21 16:44:10 | ↓ Successfully download the url to: ./paper/The user-subjective approach to personal information management systems. Journal of the American Society for Information Science and Technology, 54(9), 872–878.pdf
[INFO] | 2022/09/21 16:44:10 | Choose scihub url [0]: http://sci-hub.se


100% [..........................................................] 75413 / 75413
---------------------------
---------------------------
4 10.1016/S0005-7967(01)00136-X


[INFO] | 2022/09/21 16:44:10 | <- Request: scihub_url=http://sci-hub.se, source=DoiSource[type=doi, id=10.1016/S0005-7967(01)00136-X]
[INFO] | 2022/09/21 16:44:10 | -> Response: status_code=200, content_length=7005
[INFO] | 2022/09/21 16:44:10 | * Extracted information: {'url': 'https://moscow.sci-hub.se/1722/a1c96cb89675f57f7ee4f3102b974360/coles2003.pdf', 'title': 'Hoarding behaviors in a large college sample. Behaviour Research and Therapy, 41(2), 179–194'}
[INFO] | 2022/09/21 16:44:11 | ↓ Successfully download the url to: ./paper/Hoarding behaviors in a large college sample. Behaviour Research and Therapy, 41(2), 179–194.pdf
[INFO] | 2022/09/21 16:44:11 | Choose scihub url [0]: http://sci-hub.se


...

100% [........................................................] 456416 / 456416
21 10.1002/jclp.20795


[INFO] | 2022/09/21 16:44:38 | <- Request: scihub_url=http://sci-hub.se, source=DoiSource[type=doi, id=10.1002/jclp.20795]
[INFO] | 2022/09/21 16:44:38 | -> Response: status_code=200, content_length=6965
[INFO] | 2022/09/21 16:44:38 | * Extracted information: {'url': 'https://zero.sci-hub.se/6200/4b58f613173db55e15b08e080ba56b17/tolin2011.pdf', 'title': 'Understanding and treating hoarding  A biopsychosocial perspective. Journal of Clinical Psychology, 67(5), 517–526'}
[INFO] | 2022/09/21 16:44:38 | ↓ Successfully download the url to: ./paper/Understanding and treating hoarding  A biopsychosocial perspective. Journal of Clinical Psychology, 67(5), 517–526.pdf
[INFO] | 2022/09/21 16:44:38 | Choose scihub url [0]: http://sci-hub.se


100% [........................................................] 117411 / 117411
22 10.1016/j.psychres.2009.05.001


[INFO] | 2022/09/21 16:44:38 | <- Request: scihub_url=http://sci-hub.se, source=DoiSource[type=doi, id=10.1016/j.psychres.2009.05.001]
[INFO] | 2022/09/21 16:44:38 | -> Response: status_code=200, content_length=7059
[INFO] | 2022/09/21 16:44:38 | * Extracted information: {'url': 'https://moscow.sci-hub.se/1699/be2af3a1c18405a797f1fcdda06b7364/tolin2010.pdf', 'title': 'A brief interview for assessing compulsive hoarding  The Hoarding Rating Scale-Interview. Psychiatry Research, 178(1), 147–152'}
[INFO] | 2022/09/21 16:44:39 | ↓ Successfully download the url to: ./paper/A brief interview for assessing compulsive hoarding  The Hoarding Rating Scale-Interview. Psychiatry Research, 178(1), 147–152.pdf


100% [........................................................] 193082 / 193082
---------------------------


...now a question is how we can also access to "cross references" that is DOI's articles that reference an article with a given DOI.


We do allow searches for the counts against a DOI for cited by but at the moment there is no way in the REST API to search for cited by metadata for works, as members can only retrieve cited-by metadata for their own content.
More information can be found here: https://www.crossref.org/services/cited-by/https://github.com/CrossRef/rest-api-doc/issues/374

Not from Crossref, but from @opencitations there is COCI, the OpenCitations Index of Crossref open DOI-to-DOI references, cf. this blogpost https://opencitations.wordpress.com/2018/07/12/coci/ which allows such queries on the open part of the Crossref citation data.









It seems that "Opencitations" is the better tool.
https://github.com/opencitations
https://opencitations.net/index/coci/api/v1#/citations/{doi}


There are HTTP requests, which can be done from python.
https://opencitations.net/index/coci/api/v1/references/10.1017/jfm.2017.541
https://opencitations.net/index/coci/api/v1/citations/10.1017/jfm.2017.541

Stillman, G. (2011). Applying Metacognitive Knowledge and Strategies in Applications and Modelling Tasks at Secondary School (pp. 165–180). https://doi.org/10.1007/978-94-007-0910-2_18

Vorhölter, K. (2019). Enhancing metacognitive group strategies for modelling. ZDM Mathematics Education, 51(4), 703-716. https://doi.org/10.1007/s11858-019-01055-7

Stillman, G. A., & Galbraith, P. L. (1998). Applying mathematics with real world connections: metacognitive characteristics of secondary students. Educational Studies in Mathematics, 36(2), 157–194. https://doi.org/10.1023/A:1003246329257

Borromeo, R. (2006). Theoretical and empirical differentiations of phases in the modelling process. ZDM Mathematics Education, 38(2), 86–95. https://doi.org/10.1007/BF02655883


Kaiser, G., & Schwarz, B. (2010). Authentic Modelling Problems in Mathematics Education- Examples and Experiences. Journal für Mathematik-Didaktik, 31(1), 51–76. https://doi.org/10.1007/s13138-010-0001-3

Novoa-Muñoz F., 2019. GcCommunications in Statistics - Simulation and Computation. https://doi.org/10.1080/03610918.2019.1590598

Wedelin, D., Adawi, T., Jahan, T., & Andersson, S. (2015). Investigating and developing engineering students’ mathematical modelling and problem-solving skills. European
Journal of Engineering Education, 40(5), 557–572.
https://doi.org/10.1080/03043797.2014.987648

Novoa-Muñoz F., Jiménez-Gamero M.D., 2013. Testing for the bivariate Poisson distribution. Metrika. 77, 771-793.
https://doi.org/10.1007/s00184-013-0464-6



Meintanis S., Bassiakos Y., 2005. Goodness-of-fit tests for additively closed count models with an application to the generalized Hermite distribution. Sankhya. 67, 538–552. DOI:10.2307/25053448

    

In [89]:
DOI = "10.1017/jfm.2017.541"
DOI = '10.1007/978-94-007-0910-2_18' # Stillman; book !

DOI = 'doi.org/10.1007/s11858-019-01055-7' # Vorhölter; !
DOI = '10.1023/A:1003246329257' # Stillman, G. A., & Galbraith; ERROR
DOI = '10.1007/BF02655883' # Borromeo; - no abstract
DOI = '10.1007/s13138-010-0001-3' # Kaiser, G., & Schwarz, B.
DOI = '10.1007/BF02655885'
DOI = '10.1007/BF02212307'
DOI = '10.1080/03610918.2019.1590598' # Novoa-Muñoz; ERR: no tiene referencias
DOI = '10.1080/03043797.2014.987648'
DOI = '10.1007/s00184-013-0464-6' # Testing for the bivariate Poisson distribution; varias referencias, sin resumen reportado
# DOI = '10.2307/25053448' # Meintanis S., Bassiakos Y.; sin referencias 

DOI = '10.1016/j.chb.2019.01.037' # Natalia
# DOI = '10.22937/ijcsns.2022.22.4.37' # Felipe
doi = DOI

In [90]:
import requests
# from requests import get

HTTP_HEADERS = {"authorization": "8bd01ec8-0c5e-44fc-9b56-c7b7565dd487"}
    
def get_citing(DOI):
    API_CALL_CIT= "https://opencitations.net/index/coci/api/v1/citations/"
    API_CALL = API_CALL_CIT + DOI
    HTTP_HEADERS = {"authorization": "YOUR-OPENCITATIONS-ACCESS-TOKEN"} # You can read the FAQs and get your token here: https://opencitations.net/accesstoken 
    HTTP_HEADERS = {"authorization": "8bd01ec8-0c5e-44fc-9b56-c7b7565dd487"}
    response = requests.get(API_CALL, headers = HTTP_HEADERS)
    response_dict = json.loads(response.text)
    return response_dict
    # print(json.dumps(response_dict, sort_keys=True, indent=4))

    
def get_cited(DOI):
    API_CALL_REF = "https://opencitations.net/index/coci/api/v1/references/"
    API_CALL = API_CALL_REF + DOI
    HTTP_HEADERS = {"authorization": "8bd01ec8-0c5e-44fc-9b56-c7b7565dd487"}    
    response = requests.get(API_CALL, headers = HTTP_HEADERS)
    response_dict = json.loads(response.text)
    return response_dict
    # print(json.dumps(response_dict, sort_keys=True, indent=4))

In [91]:
response_dict = get_citing(DOI)

Figuring out the formate of the response.

In [92]:
response_dict[0]

{'author_sc': 'no',
 'journal_sc': 'no',
 'cited': '10.1016/j.chb.2019.01.037',
 'timespan': '-P0Y6M',
 'oci': '020010107073602000505020007060109080802010702-02001000106361937121711370200010937000137000307',
 'citing': '10.1177/2055207619882172',
 'creation': '2019-01'}

So, the citing url is accessible by the label "citing".

In [93]:
response_dict[0].get('citing')

'10.1177/2055207619882172'

In [94]:
len(response_dict)

10

....


In [95]:
# resumenes
from crossref.restful import Works
works = Works()

def get_abstract(doi):
    dct = works.doi(doi)
    abstract = dct.get('abstract')
    print('abstract(', doi ,'):\n', abstract,'\n')
    

# doi = '10.1016/j.csda.2022.107548'
get_abstract(doi)

abstract( 10.1016/j.chb.2019.01.037 ):
 None 



In [96]:
# show citing_
def show_citing_articles(doi):
    get_data(doi)
    response_dict = get_citing(doi)
    for k in range(0, len(response_dict)):
        doi = response_dict[k].get('citing')
        if(doi):
            r = get_citing(doi)
            print(k,': ', doi, '# cited', len(r))
            # get_data(doi) 
            # get_abstract(doi)
            # scihub_download(doi, out=out)
        else:
            print("---------------------------")
    return

def show_cited_articles(doi):
    get_data(doi)
    get_abstract(doi)
    response_dict = get_cited(doi)
    for k in range(0, len(response_dict)):
        doi = response_dict[k].get('cited')
        if(doi):
            r = get_citing(doi)
            print(k,': ', doi, '# cited', len(r))
            # get_data(doi) 
            # get_abstract(doi)
            # scihub_download(doi, out=out)
        else:
            print("---------------------------")

In [97]:
# get_cited
# doi = '10.1006/jmva.2001.2041'
show_cited_articles(doi)




title: ['Digital hoarding behaviours: Measurement and evaluation'] 
 ref_cout: 24 ; cited_count: 11 ;
 abstract: None 


abstract( 10.1016/j.chb.2019.01.037 ):
 None 

0 :  10.1007/s10862-005-3265-z # cited 63
1 :  10.1016/s0005-7967(01)00136-x # cited 127
2 :  10.1023/a:1025428631552 # cited 230
3 :  10.1002/asi.10283 # cited 67
4 :  10.1016/0005-7967(93)90094-b # cited 386
5 :  10.1016/j.cpr.2003.08.002 # cited 335
6 :  10.1016/j.psychres.2009.05.001 # cited 217
7 :  10.1002/asi.22864 # cited 38
8 :  10.1002/jclp.20795 # cited 39
9 :  10.1192/bjp.bp.113.130195 # cited 97
10 :  10.1287/orsc.2014.0945 # cited 68
11 :  10.21153/ps2015vol1no2art472 # cited 8
12 :  10.1002/meet.2011.14504801304 # cited 11
13 :  10.1016/0003-6870(88)90199-8 # cited 184
14 :  10.1016/j.chb.2018.03.031 # cited 22


In [98]:
doi = '10.1137/1138055'
get_data(doi)

title: ['On Multivariate Skewness and Kurtosis'] 
 ref_cout: 7 ; cited_count: 59 ;
 abstract: None 




In [77]:
doi = '10.1016/j.chb.2019.01.037' # Natalia
# doi = '10.1016/j.cose.2020.102003' # Felipe
show_citing_articles(doi)

title: ['Digital hoarding behaviours: Measurement and evaluation'] 
 ref_cout: 24 ; cited_count: 11 ;
 abstract: None 


0 :  10.1177/2055207619882172 # cited 1
1 :  10.1093/iwc/iwaa015 # cited 1
2 :  10.1016/j.chb.2020.106292 # cited 9
3 :  10.1016/b978-0-12-819204-7.00005-1 # cited 2
4 :  10.1016/j.ijhcs.2020.102503 # cited 8
5 :  10.1007/978-3-030-71796-4_6 # cited 0
6 :  10.3167/screen.2021.060106 # cited 0
7 :  10.1177/09610006221079357 # cited 1
8 :  10.1016/j.cose.2022.102751 # cited 0
9 :  10.1109/ddp.2019.00019 # cited 0


In [99]:
doi = '10.1016/0005-7967(93)90094-b'
show_citing_articles(doi)

title: ['The hoarding of possessions'] 
 ref_cout: 31 ; cited_count: 396 ;
 abstract: None 


0 :  10.1046/j.1365-2524.2000.00245.x # cited 236
1 :  10.1111/bjc.12212 # cited 2
2 :  10.1097/nhh.0b013e3182838847 # cited 4
3 :  10.1097/nmd.0b013e31823f678b # cited 32
4 :  10.1080/13607863.2012.678480 # cited 23
5 :  10.1080/13674676.2019.1583176 # cited 1
6 :  10.1146/annurev-clinpsy-032511-143116 # cited 54
7 :  10.1080/1556035x.2014.868725 # cited 56
8 :  10.1080/16506073.2012.738242 # cited 29
9 :  10.1080/20932685.2017.1313125 # cited 1
10 :  10.3109/00048674.2011.567243 # cited 40
11 :  10.3109/00048679709065083 # cited 21
12 :  10.3109/00952990.2010.493590 # cited 143
13 :  10.4140/tcp.n.2011.458 # cited 0
14 :  10.1017/9781108140416.024 # cited 0
15 :  10.14389/jsad.6.2_72 # cited 4
16 :  10.2466/15.pr0.116k17w0 # cited 7
17 :  10.2466/pr0.106.1.279-296 # cited 13
18 :  10.1186/s13030-019-0156-1 # cited 0
19 :  10.1037/a0032261 # cited 6
20 :  10.3389/fnbeh.2016.00030 # cited 3
21

KeyboardInterrupt: 

In [79]:
doi = '10.1016/j.ijhcs.2020.102503'
show_citing_articles(doi)

title: ['Anxious and frustrated but still competent: Affective aspects of interactions with personal information management'] 
 ref_cout: 71 ; cited_count: 9 ;
 abstract: None 


0 :  10.1108/ajim-07-2020-0208 # cited 1
1 :  10.1007/978-3-030-71796-4_6 # cited 0
2 :  10.1007/978-3-030-71796-4_8 # cited 0
3 :  10.1177/03400352211065487 # cited 0
4 :  10.1108/jd-12-2021-0236 # cited 0
5 :  10.1177/09610006221079357 # cited 1
6 :  10.1016/j.cose.2022.102751 # cited 0
7 :  10.1002/asi.24692 # cited 0


In [100]:
doi = '10.1177/0887302x11411709'
show_citing_articles(doi)

title: ['Fast Fashion and In-Store Hoarding'] 
 ref_cout: 52 ; cited_count: 46 ;
 abstract: <jats:p> In-store hoarding refers to the practice of taking possession of an item and keeping it for themselves while shopping although they are not sure whether to buy or not. We examined theoretical links among in-store behavior, hedonic shopping value, and repatronage intention and a moderating role of fashion innovativeness in the impacts of the three fast fashion factors (perceived perishability, scarcity, and low price) on in-store hoarding. The model was tested using female shoppers from two leading fast fashion retailers in the U.S. We found that while perceived perishability was a stronger driver of in-store hoarding for the high innovativeness group, perceived scarcity was a stronger driver of this act for the low innovativeness group. In-store hoarding and perceived perishability tend to produce positive retail outcomes by increasing hodonic shopping value and repatronage intentions. 

In [101]:
doi = '10.1016/j.jclepro.2017.06.128'
show_citing_articles(doi)

title: ['Life cycle assessment of clothing libraries: can collaborative consumption reduce the environmental impact of fast fashion?'] 
 ref_cout: 34 ; cited_count: 120 ;
 abstract: None 


0 :  10.1108/jfmm-03-2020-0044 # cited 5
1 :  10.1016/j.jclepro.2021.126245 # cited 34
2 :  10.1016/j.jclepro.2021.127080 # cited 6
3 :  10.1016/b978-0-12-821483-1.00009-7 # cited 1
4 :  10.1016/j.spc.2021.07.030 # cited 6
5 :  10.1007/978-981-16-2414-8_1 # cited 1
6 :  10.1016/j.jclepro.2021.128648 # cited 2
7 :  10.1080/27658511.2021.1975916 # cited 0
8 :  10.1016/j.clrc.2021.100033 # cited 1
9 :  10.4018/978-1-7998-7545-1.ch007 # cited 1
10 :  10.1021/acssuschemeng.9b01742 # cited 23
11 :  10.3389/fsufs.2021.787281 # cited 1
12 :  10.4018/978-1-6684-2523-7.ch015 # cited 0
13 :  10.1108/scm-05-2021-0240 # cited 0
14 :  10.21511/ppm.19(4).2021.01 # cited 1
15 :  10.1038/s41893-021-00775-7 # cited 1
16 :  10.3934/ctr.2021004 # cited 5
17 :  10.1016/j.clrc.2021.100039 # cited 0
18 :  10.1016/j.rser.2

KeyboardInterrupt: 

In [102]:
doi = '10.1016/j.jclepro.2020.121026'
show_citing_articles(doi)



title: ['Waste not want not: Behavioural intentions toward garment life extension practices, the role of damage, brand and cost on textile disposal'] 
 ref_cout: 26 ; cited_count: 11 ;
 abstract: None 


0 :  10.1007/s10668-020-00844-5 # cited 30
1 :  10.3390/su12219151 # cited 10
2 :  10.1080/00405000.2020.1848114 # cited 1
3 :  10.1016/j.jclepro.2021.126516 # cited 6
4 :  10.1002/bse.2821 # cited 11
5 :  10.1080/00405000.2021.1954428 # cited 1
6 :  10.1108/jfmm-09-2020-0192 # cited 3
7 :  10.17230/map.v10.i19.03 # cited 0
8 :  10.4018/978-1-6684-2523-7.ch015 # cited 0
9 :  10.3390/su14010487 # cited 0
10 :  10.1080/17543266.2022.2072958 # cited 0


In [103]:
doi = '10.1007/s10668-020-00844-5'
show_citing_articles(doi)

title: ['Drivers to green consumption: a systematic review'] 
 ref_cout: 178 ; cited_count: 39 ;
 abstract: None 


0 :  10.1007/s10668-020-01168-0 # cited 1
1 :  10.3390/en14102940 # cited 0
2 :  10.3390/su13116219 # cited 13
3 :  10.1007/s10668-021-01510-0 # cited 1
4 :  10.1007/s43615-021-00057-6 # cited 3
5 :  10.1002/bse.2889 # cited 4
6 :  10.1080/03772063.2021.1906336 # cited 1
7 :  10.4018/978-1-7998-8678-5.ch015 # cited 0
8 :  10.3390/en14175551 # cited 2
9 :  10.3390/agriculture11111050 # cited 4
10 :  10.3390/en14217061 # cited 4
11 :  10.1080/08911762.2021.1983686 # cited 1
12 :  10.3390/su132312988 # cited 0
13 :  10.3390/pr10020395 # cited 0
14 :  10.1177/18393349211069152 # cited 8
15 :  10.3390/en15031027 # cited 2
16 :  10.3390/su14020689 # cited 1
17 :  10.1108/ijshe-11-2020-0459 # cited 0
18 :  10.3390/su14073999 # cited 1
19 :  10.1007/s11356-022-20024-4 # cited 1
20 :  10.1016/j.spc.2022.04.012 # cited 0
21 :  10.3390/en15062077 # cited 2
22 :  10.1016/j.resconrec.

In [104]:
doi = '10.3390/su13116219'
show_citing_articles(doi)

title: ['Determinants That Influence Green Product Purchase Intention and Behavior: A Literature Review and Guiding Framework'] 
 ref_cout: 173 ; cited_count: 18 ;
 abstract: <jats:p>Environmental deterioration brought about by consumers’ non-feasible utilization pattern is putting a pressure on the environment and is obstructing sustainable development. To hinder this impact and stimulate a more sustainable economic growth, one solution is to lessen or move utilization patterns from conventional products to eco-friendly products. The authors conducted a review study of green purchase behavioral research across the 6-year period from 2015 to 2021, identifying 108 studies that met our inclusion criteria. The current review distinguishes different pervasive facilitators, motives, and obstacles influencing consumers’ decision-making process towards environmentally friendly products, and it gives potential clarifications for contradictions found in green purchase behavior (GPB). The paper 

In [ ]:
for k in range(0, len(response_dict)):
    doi = response_dict[k].get('citing')
    if(doi):
        print(k,': ', doi)
        get_abstract(doi)
        # scihub_download(doi, out=out)
    else:
        print("---------------------------")

It works for both citations and references.


Here some possiblities for queries with "crossrefs:

In [ ]:
works = Works()
works.query('design thinking').url

In [ ]:
works.query('design thinking').filter(from_online_pub_date='2020').count()

In [ ]:
wq=works.query('design thinking').filter(from_online_pub_date='2020')


Other demos
* https://pypi.org/project/crossrefapi/1.0.3/
* https://github.com/CrossRef/rest-api-doc/blob/master/demos/crossref-api-demo.ipynb
